### 利用用户行为数据
#### 用户行为数据
用户行为在个性化推荐系统中一般分为两种，显性反馈行为和隐性反馈行为。<br>
- 显性反馈行为：包括用户明确表示对物品喜好的行为，评分，喜欢和不喜欢等。
- 隐性反馈行为：指代那些不能明确反应用户喜好的行为，如页面浏览行为 <br>
数据集分类：
- 无上下文信息的隐形反馈数据集
- 无上下文信息的显性反馈数据集
- 有上下文信息的隐性反馈数据集
- 有上下文信息的显性反馈数据集

#### 用户行为分析

##### 长尾分布
$$
    f(x) = \alpha X^k
$$
研究发现，互联网上的很多数据分布都满足一种称为PowerLaw的分布。

仅仅基于用户行为数据设计的推荐算法一般称为协同过滤算法。协同过滤算法包括，基于邻域的方法、隐语义模型、基于图的随机游走方法。其中，基于邻域的方法应用最为广泛，基于邻域的方法应用最为广泛，主要包括：1. 基于用户的协同过滤算法 2. 基于物品的协同过滤算法。

#### 实验
数据集：MovieLens数据集 <br>
评测指标：召回率、准确率、覆盖率、新颖度 <br>
指标计算算法：

In [136]:
# precision
def Precision(train,test,N,recommand_func=recommand):
    hit,all_recommand = 0,0
    for user in train.keys():
        real = test[user] if user in test else []
        recommand_items = getRecommendation(user,N,recommand_func)
        for item in recommand_items:
            if item[0] in real:
                hit += 1
        all_recommand += N
    print('hit:%d'% hit)
    return hit / all_recommand
# recall
def Recall(train,test,N,recommand_func=recommand):
    hit,recommands = 0,0
    for user in train.keys():
        real = test[user] if user in test else []
        recommand_items = getRecommendation(user,N,recommand_func)
        for item in recommand_items:
            if item[0] in real:
                hit += 1
        recommands += len(real)
    print('hit:{},recommands:{}'.format(hit,recommands))
    return hit / recommands 
# coverage
def Coverage(train,test,N):
    recommand_item = set()
    all_item = set()
    for user in train.keys():
        all_item.update(train[user].keys())    
        items = getRecommendation(user,N)
        recommand_item.update(items)
    return len(recommand_item) / len(all_item)
# popularity
def Popularity():
    pass

In [30]:
import random
def load_data(users_path,movies_path,rating_path):
    
    def read_users():
        """(user_id:(gender,age,occupation))"""
        users = {}
        with open(users_path,'r') as f:
            users = {line.split("::")[0]:line.split("::")[1:-1] for line in f.readlines()}
        return users
    def read_movies():
        """(movie_id:(title,genres))"""
        movies = {}
        with open(movies_path,'r') as f:
            movies = {line.split("::")[0]: line.split("::")[1:] for line in f.readlines()}
        return movies
    def read_ratings():
        """{user_id:{movie_id: rating}}"""
        ratings = dict()
        with open(rating_path, 'r') as f:
            for line in f.readlines():
                user,movie,rating,_ = line.split('::')
                ratings.setdefault(user,{})
                ratings[user][movie] = int(rating)
        return ratings
    return read_users(),read_movies(),read_ratings()
def split_train_test(data,M,k,seed):
    """M折交叉验证"""
    train_set = dict()
    test_set = dict()
    random.seed(seed)
    test_count,train_count = 0,0
    for user,movie_info in data.items():
        for movie,rating in movie_info.items():
            if random.randint(0,M) == k:
                test_set.setdefault(user,{})
                test_set[user][movie] = int(rating)
                test_count += 1
            else:
                train_set.setdefault(user,{})
                train_set[user][movie] = int(rating)
                train_count += 1
    print(test_count,train_count)
    return train_set,test_set
USERS_PATH = '../dataset/ml-1m/users.dat'
MOVIES_PATH = '../dataset/ml-1m/movies.dat'
RATINGS_PATH = '../dataset/ml-1m/ratings.dat'
users,movies,ratings = load_data(USERS_PATH,MOVIES_PATH,RATINGS_PATH)
train_set,test_set = split_train_test(ratings,5,1,1)

166907 833302


##### 基于邻域的算法
基于邻域的算法是推荐系统中，最基本的算法，可分为两大类，**基于用户的协同过滤算法**以及**基于物品的协同过滤算法**。
###### 基于用户的协同过滤算法
1. 找到和目标用户兴趣相似的用户集合
2. 找到这个集合中的用户喜欢的，且目标用户未接触过的物品
$$
   w[u][v] = \frac{|N(u)\cap N(v)|}{\sqrt{|N(u)||N(v)|}}
$$

In [40]:
import math
def user_similarity_cos(userA,userB):
    """用户余弦相似度"""
    count = 0
    for movie in userA.keys():
        if movie in userB.keys():
            count += 1
    return count / math.sqrt((len(userA)*len(userB)))
def user_similarity_jaccard(userA,userB):
    """用户Jaccard相似度"""
    count = 0
    for movie in userA.keys():
        if movie in userB.keys():
            count += 1
    return count /(len(userA) + len(userB) - count)
user_similarity_cos(train_set['1'],train_set['2']),user_similarity_jaccard(train_set['1'],train_set['2'])

(0.07042952122737638, 0.03289473684210526)

###### 倒排表优化
如果在每两个用户之间进行对比，将花费大量时间，同时，很多用户之间并没有对同样对物品产生过行为，这样算法将很多时间都浪费在了计算这种用户之间的相似度上了。<br>
**倒排表**:相比于用户->物品这样的索引格式，我们将物品->用户列表做反向索引的数据结构，称为倒排表。<br>
通过建立倒排表，可以只针对位于同一物品下用户列表的用户进行处理。

In [41]:
def inverse_table(data):
    """电影->用户，倒排表"""
    movies_inverse_table = {}
    for user,movies in data.items():
        for movie in movies.keys():
            movies_inverse_table.setdefault(movie,set())
            movies_inverse_table[movie].add(user)
    return movies_inverse_table

In [43]:
train_table = inverse_table(train_set)

In [64]:
from collections import defaultdict
def user_cf_matrix(table):
    user_matrix = {}
    N = defaultdict(int) # 用户活跃度
    # 构建用户之间接触过相同movie数量的矩阵
    for movie,userlist in table.items():
        for u in userlist:
            N[u] += 1
            for v in userlist:
                if u == v:
                    continue
                user_matrix.setdefault(u,defaultdict(int))
                user_matrix[u][v] += 1
    # 根据矩阵，计算用户之间的相似度矩阵
    similarity_matrix = defaultdict(dict)
    for u,related_users in user_matrix.items():
        for v,related_num in related_users.items():
            fenmu = math.sqrt(N[u]*N[v])
            similarity_matrix[u][v] = related_num / fenmu
    return similarity_matrix

In [66]:
similarity_matrix = user_cf_matrix(train_table)

In [69]:
from operator import itemgetter
def recommand(user,data,similarity_matrix,K):
    rank = defaultdict(int)
    used_item = data[user]
    r_vi = 1 #代表用户v对物品的感兴趣程度
    for v,similarity in sorted(similarity_matrix[user].items(),key=itemgetter(1),reverse=True)[0:K]:
        for movie,rating in data[v].items():
            if movie in used_item:
                continue
            rank[movie] += similarity * r_vi
    return rank

In [72]:
recommand_items = recommand('1',train_set,similarity_matrix,5)

In [92]:
sorted(recommand_items.items(),key=lambda item:item[1],reverse=True)

[('595', 1.5248196736913286),
 ('593', 1.2402292038277192),
 ('2096', 1.2383737240755557),
 ('2571', 1.1649559276584556),
 ('2396', 1.1649559276584556),
 ('2078', 0.9539464461915693),
 ('2080', 0.9539464461915693),
 ('2137', 0.9539464461915693),
 ('1028', 0.9539464461915693),
 ('2081', 0.8805286497744692),
 ('1907', 0.8805286497744692),
 ('364', 0.8805286497744692),
 ('1393', 0.8803654577948459),
 ('34', 0.8785099780426824),
 ('1210', 0.8785099780426824),
 ('3578', 0.6442910239168596),
 ('1407', 0.5959381799108596),
 ('356', 0.5959381799108596),
 ('480', 0.5959381799108596),
 ('2054', 0.594082700158696),
 ('2', 0.594082700158696),
 ('2085', 0.594082700158696),
 ('3034', 0.594082700158696),
 ('904', 0.594082700158696),
 ('1136', 0.594082700158696),
 ('3157', 0.594082700158696),
 ('3159', 0.594082700158696),
 ('1580', 0.594082700158696),
 ('709', 0.594082700158696),
 ('1265', 0.594082700158696),
 ('2628', 0.594082700158696),
 ('2394', 0.594082700158696),
 ('1032', 0.594082700158696),
 ('

In [113]:
def getRecommendation(user,N):
    recommand_items = recommand(user,train_set,similarity_matrix,5)
    return sorted(recommand_items.items(),key=lambda item:item[1],reverse=True)[:N]

In [109]:
Precision(train_set,test_set,10),Recall(train_set,test_set,10)

hit:12581
hit:12581,recommands:166907


(0.20829470198675495, 0.07537730592485636)

通过增加对热门物品对惩罚手段，可以提高推荐的性能。
$$
w_{uv} = \frac{\sum_{i\in N(u)\cap N(v)}{\frac{1}{log{1+|N(i)|}}}}{\sqrt{|N(u)||N(v)|}}
$$

In [144]:
def user_iif_matrix(table):
    user_matrix = {}
    N = defaultdict(int)
    # 构建用户之间接触过相同movie数量的矩阵
    for movie,userlist in table.items():
        for u in userlist:
            N[u] += 1
            for v in userlist:
                if u == v:
                    continue
                user_matrix.setdefault(u,defaultdict(int))
                user_matrix[u][v] += 1 / math.log(1 + len(userlist)) # 对热门物品进行惩罚
    # 根据矩阵，计算用户之间的相似度矩阵
    similarity_matrix = defaultdict(dict)
    for u,related_users in user_matrix.items():
        for v,related_cuv in related_users.items():
            fenmu = math.sqrt(N[u]*N[v])
            similarity_matrix[u][v] = related_cuv / fenmu
    return similarity_matrix

In [111]:
similarity_iif_matrix = user_iif_matrix(train_table)

In [114]:
similarity_matrix = similarity_iif_matrix
Precision(train_set,test_set,10),Recall(train_set,test_set,10)

hit:12254
hit:12254,recommands:166907


(0.20288079470198675, 0.07341813105501867)

###### 基于物品的协同过滤算法
1. 计算物品之间的相似度
2. 根据物品相似度和用户的历史行为数据给用户生成推荐列表
$$
w_{i,j} = \frac{|N(i)\cap N(j)|}{\sqrt(|N(i)||N(j)|)}
$$

In [121]:
def item_inverse_table(data):
    """user->item"""
    return data

In [145]:
def item_cf_matrix(table):
    item_matrix = {}
    N = defaultdict(int) # 物品热门度 
    # 构建共现矩阵
    for user,movie_info in table.items():
        for movie in movie_info.keys():
            N[movie] += 1
            for movie_same in movie_info.keys():
                if movie_same == movie:
                    continue
                item_matrix.setdefault(movie,defaultdict(int))
                item_matrix[movie][movie_same] += 1
    # 计算物品之间相似度矩阵
    similarity_matrix = defaultdict(dict)
    for movie,movie_sames in item_matrix.items():
        for movie_same,related_num in movie_sames.items():
            similarity_matrix[movie][movie_same] = related_num / math.sqrt(N[movie]*N[movie_same])
    return similarity_matrix

In [146]:
table = item_inverse_table(train_set)
similarity_matrix = item_cf_matrix(table)

In [153]:
from operator import itemgetter
def recommand_by_item(user,data,similarity_matrix,K):
    rank = defaultdict(int)
    used_item = data[user]
    r_vi = 1 #代表用户v对物品的感兴趣程度
    for item, score in used_item.items():
        for v,similarity in sorted(similarity_matrix[item].items(),key=itemgetter(1),reverse=True)[0:K]:
            if v in used_item:
                continue
            rank[v] += score * similarity
    return rank

In [154]:
recommand_items = recommand_by_item('1',train_set,similarity_matrix,10)

In [156]:
sorted(recommand_items.items(),key=lambda item:item[1],reverse=True)

[('595', 20.667303203072034),
 ('1196', 17.438339996566377),
 ('1270', 16.84439260419705),
 ('2080', 13.648296284713194),
 ('2858', 13.048566902842724),
 ('364', 12.480164565823191),
 ('1968', 12.180239597293857),
 ('318', 11.395848902585536),
 ('2096', 11.390103475161045),
 ('1198', 10.61989190267215),
 ('1265', 10.61544768700103),
 ('2716', 10.088916126452377),
 ('593', 9.754533657279017),
 ('1307', 9.58130205238222),
 ('596', 9.544036622874486),
 ('2571', 9.410760851125342),
 ('2087', 9.338490902789605),
 ('2174', 8.450845090967345),
 ('1907', 8.295443467184407),
 ('2396', 7.253630711381443),
 ('1032', 7.065593701627355),
 ('1028', 6.993585818237502),
 ('1394', 6.352643185881094),
 ('1302', 6.064104371822799),
 ('1079', 6.052024252705675),
 ('34', 6.042622532868319),
 ('1240', 6.030320869280478),
 ('1210', 6.018200879646214),
 ('1704', 5.960617488061258),
 ('2078', 5.857018867989942),
 ('1282', 5.77913395684892),
 ('296', 5.640536988087691),
 ('356', 5.44393167867393),
 ('3448', 5.3

In [150]:
test_set['1']

{'1193': 5,
 '595': 5,
 '1270': 5,
 '1907': 4,
 '150': 5,
 '2692': 4,
 '1028': 5,
 '1207': 4}

In [157]:
def getRecommendation(user,N,recommand=recommand):
    recommand_items = recommand(user,train_set,similarity_matrix,5)
    return sorted(recommand_items.items(),key=lambda item:item[1],reverse=True)[:N]

In [158]:
Precision(train_set,test_set,10,recommand_func=recommand_by_item),Recall(train_set,test_set,10,recommand_func=recommand_by_item)

hit:15608
hit:15608,recommands:166907


(0.2584105960264901, 0.09351315403188602)

In [139]:
train_set['1']

{'661': 3,
 '914': 3,
 '3408': 4,
 '2355': 5,
 '1197': 3,
 '1287': 5,
 '2804': 5,
 '594': 4,
 '919': 4,
 '938': 4,
 '2398': 4,
 '2918': 4,
 '1035': 5,
 '2791': 4,
 '2687': 3,
 '2018': 4,
 '3105': 5,
 '2797': 4,
 '2321': 3,
 '720': 3,
 '527': 5,
 '2340': 3,
 '48': 5,
 '1097': 4,
 '1721': 4,
 '1545': 4,
 '745': 3,
 '2294': 4,
 '3186': 4,
 '1566': 4,
 '588': 4,
 '783': 4,
 '1836': 5,
 '1022': 5,
 '2762': 4,
 '1': 5,
 '1961': 5,
 '1962': 4,
 '260': 4,
 '1029': 5,
 '2028': 5,
 '531': 4,
 '3114': 4,
 '608': 4,
 '1246': 4}

In [160]:
def get_reason(item_id,matrix,K):
    res = []
    movie_name = movies[item_id][0] 
    for same_item,similarity in sorted(similarity_matrix[item_id].items(),key=itemgetter(1),reverse=True)[0:K]:
        res.append((movies[same_item][0],similarity))
    return res

In [161]:
get_reason('588',similarity_matrix,5)

[('Beauty and the Beast (1991)', 0.5596452972866592),
 ('Lion King, The (1994)', 0.5512103498139401),
 ('Toy Story (1995)', 0.5155668086460745),
 ('Little Mermaid, The (1989)', 0.4959528746194235),
 ("Bug's Life, A (1998)", 0.447737936213113)]

在之前的代码中，所有用户对于物品之间相似度的贡献都是相同的，但当考虑到用户活跃度时，贡献其实是不相同的。<br>
IUF(Inverse User Frequence)：活跃度对物品相似度贡献应该小于不活跃的用户。
$$
w_{i,j} = \frac{\sum_{u \in N(i)\cap N(j)}{\frac{1}{log1+|N(u)|}}}{\sqrt{|N(i)||N(j)|}}
$$

In [162]:
def item_iuf_matrix(table):
    item_matrix = {}
    N = defaultdict(int) # 物品热门度 
    # 构建共现矩阵
    for user,movie_info in table.items():
        for movie in movie_info.keys():
            N[movie] += 1
            for movie_same in movie_info.keys():
                if movie_same == movie:
                    continue
                item_matrix.setdefault(movie,defaultdict(int))
                item_matrix[movie][movie_same] += 1 / math.log(1 + len(movie_info.keys()) * 1.0)
    # 计算物品之间相似度矩阵
    similarity_matrix = defaultdict(dict)
    for movie,movie_sames in item_matrix.items():
        for movie_same,related_num in movie_sames.items():
            similarity_matrix[movie][movie_same] = related_num / math.sqrt(N[movie]*N[movie_same])
    return similarity_matrix

In [170]:
def hit(train,test,N,recommand_func=recommand):
    hit = 0
    for user in train.keys():
        real = test[user] if user in test else []
        recommand_items = getRecommendation(user,N,recommand_func)
        for item in recommand_items:
            if item[0] in real:
                hit += 1
    print('hit:%d'% hit)
    return hit
# precision
def Precision(train,hit,N):
    print('sum:%d' % (len(train.keys()) * N))
    return hit / len(train.keys()) * N
# recall
def Recall(train,test,hit):
    recommands = 0
    for user in train.keys():
        real = test[user] if user in test else []
        recommands += len(real)
    print('sum_real: %d' % recommands)
    return hit / recommands 

In [168]:
similarity_matrix = item_iuf_matrix(table)
hit_total = hit(train_set,test_set,10,recommand_func=recommand_by_item)
hit_total

hit:16186


16186

In [171]:
Precision(train_set,hit_total,10),Recall(train_set,test_set,hit_total)

sum:60400
sum_real: 166907


(26.79801324503311, 0.09697616037673674)

###### 归一化优化
由于不同类之间物品相似度不同，如A类物品之间，相似度为0.5，B类物品相似度为0.7，那么在计算相似度矩阵时，B类物品的相似度值会整体略高于A类物品。<br>
当对一个持有同等数目A类物品和B类物品的用户进行推荐时，因为B类相似度值较高，因此在计算rank时，会排于A类物品前面，因此推荐结果会出现B类物品数目>A类物品的情况。但我们明白，A、B类物品应该是相近的。<br>
通过矩阵除以其最大值，进行归一化，对该类情况进行优化。

In [184]:
def norm_matrix(matrix):
    for movie,sim_items in matrix.items():
        max_value = max(sim_items.values())
        matrix[movie] = {key: value/max_value for key,value in sim_items.items()}

In [185]:
norm_matrix(similarity_matrix)

AttributeError: 'NoneType' object has no attribute 'items'